# Feature Points

To draw the feature points, I obtain each individual feature point by selecting it using the *face* parameter that is passed into the **drawFeaturePoints** function.  I was able to analyze this object by setting a breakpoint within the function while running the application and inspecting the face parameter. Using the canvas context object I drew circles for each feature point using the *x* and *y* member variables of the featurepoint object.

~~~~~
function drawFeaturePoints(canvas, img, face) {
  // Obtain a 2D context object to draw on the canvas
  var ctx = canvas.getContext('2d');
  
  ctx.fillStyle = 'red';
  ctx.strokeStyle = 'blue';
  // Loop over each feature point in the face
  for (var id in face.featurePoints) {
    var featurePoint = face.featurePoints[id];
    
    ctx.beginPath();
    ctx.arc(featurePoint.x,featurePoint.y,3,0,2*Math.PI);
    ctx.stroke();
  }
}
~~~~~

# Dominant Emoji

To display the dominant emoji, I obtained this from the face parameter passed into the function. I randomly chose the first feature point as the reference point for where I wanted to draw the dominant emoji. Once I noticed that the first feature point was located on the left side of the face displayed, I decided to move it to the left by subtracting 10 from the feature point's x coordinate.  This placed it at a comformatable location that is easily visible.

~~~~
function drawEmoji(canvas, img, face) {
  // Obtain a 2D context object to draw on the canvas
  var ctx = canvas.getContext('2d');
  ctx.font = '30px serif';
  ctx.fillText(face.emojis.dominantEmoji,face.featurePoints[0].x -10,face.featurePoints[0].y);
}
~~~~

# Mimic Game

In implementing the Mimic Game, my first objective was defining the **update** function.  My initial goal was to display the target emoji and have the score increment if the dominantEmoji matched the target emoji.  Once I had this mechanism working, I focused on getting the scoring system correct.  This took some experimentation as I had to get back accustomed to the fact that the update function was being constantly called when a face was detected (due to the event listener). 

~~~~
function update(face,timestamp){
  
  var emoji_target = emojis[emoji_face % emojis.length];
  var audio_file = correct_audio[correct_audio_index % correct_audio.length];
  setTargetEmoji(emoji_target);
  
  if(toUnicode(face.emojis.dominantEmoji) == emoji_target){
    var audio = new Audio(audio_file);
    audio.play();
    last_update_time = 0;
    emoji_correct = true;
    correct++;
    correct_audio_index++;
  }
}
~~~~

This led me to the final task which was getting the timing and synchronization correct.  I chose to maintain a second timestamp variable and conditionally call the **update** function based on a time difference or whether the target emoji was mimic correctly.
 
 
~~~~
if(timestamp - last_update_time > 8 || emoji_correct){
      emoji_face = Math.floor(Math.random() * 34);
      var current_correct = correct;
      last_update_time = timestamp;
      total++; 
      if(!emoji_correct){
        var audio_file = incorrect_audio[incorrect_audio_index % incorrect_audio.length];
        var audio = new Audio(audio_file)
        audio.play();
        incorrect_audio_index++;
      }
      emoji_correct = false;
    }

    update(faces[0],timestamp);
    setScore(correct,total);
  }
~~~~
To put the final touches on the game, I decided to play some audio based on whether the emoji was mimic correctly or not.  I decided to use some soundbites from one of my favorite shows [Rick and Morty](http://www.adultswim.com/videos/rick-and-morty/).  I obtained the wav files from [here](https://peal.io/soundboards/rick-and-morty).  I inspected the speaker element objects from that website and copied the element that held the wav file source location.  Luckily, after I navigated to the source location, The wav file was downloadable.  So the game will play some audio based on whether the dominant emoji is correct or incorrect.


One thing that I would like to note is that the Affectiva API doesn't work particularly well on dark faces such as mine. It would pick up the pink of my inner lip as my tongue being out and when I actually stuck my tongue out, it couldn't recogninze anything at all.  I also had a difficult time detecting my face with my glasses on. I eventually took my glasses off when testing to get better results. I spent most of the time trying to get the API to recognize my face more than anything.